In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

/home/mazor/nn/nn/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random

def set_seed(seed):
    """
    Set the seed for reproducibility.
    """
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    np.random.seed(seed)  # Set the seed for numpy for reproducibility
    random.seed(seed)  # Set Python random seed
    # Ensures that CUDA operations are deterministic
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False

# Example: Setting the seed to 42
set_seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

batch_size = 32
# Transform to convert images to PyTorch tensors and normalize them
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data for MNIST
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

Using device: cuda


In [4]:
all_images = []
all_indices = []

for i, (images, labels) in enumerate(trainloader):
    all_images.extend(images)
    all_indices.extend(range(i * len(images), (i + 1) * len(images)))

# Convert the list to a NumPy array
all_images = torch.stack(all_images)
all_indices = torch.tensor(all_indices)



# Function to display an image given its index
def imshow_by_index(index):
    img = all_images[index] / 2 + 0.5  # Unnormalize
    plt.imshow(img.permute(1, 2, 0))  # Convert from Tensor image
    plt.show()

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        # Assuming MNIST images are 1x28x28 (channels x width x height)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2) # Output: 32x28x28
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2) # Output: 64x14x14
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # Output: 64x7x7 after pooling
        
        # Flatten: 64*7*7 = 3136 features
        self.fc1 = nn.Linear(3136, 128) # Fully connected layer
        
        self.fc2 = nn.Linear(128, 64) # Additional fully connected layer
        self.fc3 = nn.Linear(64, 10) # Final layer for classifying into 10 classes
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        # Flatten the output for the fully connected layers
        x = x.view(x.size(0), -1)
        
        x1 = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x1))
        x = self.fc3(x)
        return x


In [6]:
# Initialize the network and move it to the device
net = MNISTNet().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training loop
def train(net, trainloader, device, criterion, optimizer, epochs=5):
    net.train()
    for epoch in range(epochs):  
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}")

# Testing loop
def test(net, testloader, device):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %f %%' % (100 * correct / total))

# Train and test the network
train(net, trainloader, device, criterion, optimizer)


Epoch 1, Loss: 0.14668731273297453
Epoch 2, Loss: 0.048709439629817886
Epoch 3, Loss: 0.033210775474544305
Epoch 4, Loss: 0.02455204180046809
Epoch 5, Loss: 0.019545637330693293


In [7]:
test(net, testloader, device)

Accuracy of the network on the 10000 test images: 99.020000 %


In [9]:
class TrainableNode(nn.Module):
    def __init__(self, input_features, output_features):
        super(TrainableNode, self).__init__()
        self.linear = nn.Linear(input_features, output_features)
        self.is_explainable = True
    def forward(self, x):
        return self.linear(x)

# Step 1: Add the new layer
net.explainable_node = TrainableNode(128, 1).to(device)

# Step 2: Dynamically override the forward method
def new_forward(self, x, return_explainable=False):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    
    # Flatten the output for the fully connected layers
    x = x.view(x.size(0), -1)
    
    x1 = F.relu(self.fc1(x))
    
    if return_explainable:
        explainable_output = self.explainable_node(x1).squeeze()
    
    x = F.relu(self.fc2(x1))
    x = self.fc3(x)
    
    if return_explainable:
        return x, explainable_output
    return x

# Assign the new forward method to the net instance
net.forward = new_forward.__get__(net, MNISTNet)

# Optionally, if you want to directly use the explainable forward method without affecting the original forward method
def forward_with_explainable(self, x):
    # This example assumes you want to pass the original input through the explainable node directly
    # Adjust accordingly if you meant something different
    explainable_output = self.explainable_node(x).squeeze()
    return explainable_output

# Assign this method as well if needed
net.forward_with_explainable = forward_with_explainable.__get__(net, MNISTNet)


In [10]:
class ExplainableNetwork:
    def __init__(self, model):
        self.model = model
        self.activations_per_input = {}
        self.gradients_per_input = {} 
        self.batch_counter = 0
        self.layer_names = []
        self.explainable_module_names = set()
        explainable_module_names = set()
        for name, module in model.named_modules():
            if getattr(module, 'is_explainable', False):
                explainable_module_names.add(name)
        for name, module in model.named_modules():
            if name and not any(part in explainable_module_names for part in name.split('.')):
                self.layer_names.append(name)
            elif name:
                self.explainable_module_names.add(name)

        self.expected_outputs = []
        self.predicted_outputs = []

        self.hook_handles = [] 
        
        self.enable_hooks()
        
    # def getNonExpLayers(model):
    #     layers = []
    #     for name, module in model.named_modules():
    #         if name != '' and not isinstance(module, TrainableNode):
    #             layers.append(name)
    #     return layers

    # Register hooks for each layer
    def enable_hooks(self):
        for name, module in self.model.named_modules():
            if name in self.explainable_module_names:  # Skip the explainable_node by its name
                continue
            # Check if the module is a type that we want to hook (Conv2d, Linear, or pooling layers)
            if isinstance(module, (torch.nn.Linear, torch.nn.Conv2d, torch.nn.MaxPool2d)):
                forward_handle = module.register_forward_hook(self.create_forward_hook(name))
                backward_handle = module.register_full_backward_hook(self.create_backward_hook(name))
                self.hook_handles.append(forward_handle)
                self.hook_handles.append(backward_handle)
    def disable_hooks(self):
        for handle in self.hook_handles:
            handle.remove()
        self.hook_handles = []  # Clear the list after removing all hooks

    def create_forward_hook(self, layer_name):
        def forward_hook(module, input, output):
            self.store_activations(layer_name, output)
        return forward_hook

    def create_backward_hook(self, layer_name):
        def backward_hook(module, grad_input, grad_output):
            self.store_gradients(layer_name, grad_output[0])
        return backward_hook
    def store_activations(self, layer_name, output):
        self._store_per_input(self.activations_per_input, layer_name, output)

    def store_gradients(self, layer_name, gradient):
        # Store gradients for the last epoch
        batch_size = gradient.size(0)
        for i in range(batch_size):
            input_index = self.batch_counter * batch_size + i
            if input_index not in self.gradients_per_input:
                self.gradients_per_input[input_index] = {}
            self.gradients_per_input[input_index][layer_name] = gradient[i].detach().cpu().numpy()
    
            # Increment the backward batch counter only after processing the first layer (which is the last layer in the forward pass)


    def _store_per_input(self, storage_dict, layer_name, tensor):
        batch_size = tensor.size(0)
        for i in range(batch_size):
            input_index = self.batch_counter * batch_size + i
            if input_index not in storage_dict:
                storage_dict[input_index] = {}
            storage_dict[input_index][layer_name] = tensor[i].detach().cpu().numpy()

        if layer_name == self.layer_names[-1]:
            self.batch_counter += 1

    def get_activations(self, input_index):
        return self.activations_per_input.get(input_index, None)

    def get_gradients(self, input_index):
        return self.gradients_per_input.get(input_index, None)

    def print_structure(self):
        print("Explainable Network Structure and Data Summary:")

        # Check if there is any data collected
        if not self.activations_per_input:
            print("No activations data collected.")
            return

        if not self.gradients_per_input:
            print("No gradients data collected.")
            return

        # Print information about activations
        print("\nActivations:")
        first_input_activations = next(iter(self.activations_per_input.values()))
        for layer_name, activations in first_input_activations.items():
            num_nodes = len(activations)
            num_data_points = len(self.activations_per_input)
            print(f"Layer {layer_name}: {num_nodes} nodes, {num_data_points} data points each")

        # Print information about gradients
        print("\nGradients:")
        first_input_gradients = next(iter(self.gradients_per_input.values()))
        for layer_name, gradients in first_input_gradients.items():
            num_nodes = len(gradients)
            num_data_points = len(self.gradients_per_input)
            print(f"Layer {layer_name}: {num_nodes} nodes, {num_data_points} data points each")
            
    # FIXME: Add functionality for first layer
    def get_node_input(self, layer_name, node_index, image_index):
        if layer_name != self.layer_names[0] and layer_name in self.layer_names:
            # For subsequent layers, we use activations from the previous layer
            prev_layer_name = self.get_previous_layer_name(layer_name)
            prev_activations = self.activations_per_input[image_index][prev_layer_name]
        
            # Ensure the prev_activations is a 1D array for proper multiplication
            if prev_activations.ndim > 1:
                prev_activations = prev_activations.flatten()

        return prev_activations

    def find_extreme_activations(self, std_dev_threshold=3, label_digit=0, pred_digit=0):
            """
            Find inputs where node activations are more than +3 or less than -3 standard deviations from the mean.

            Args:
            std_dev_threshold (float): The number of standard deviations to use as a threshold for extreme activations.

            Returns:
            dict: A dictionary containing two keys ('high' and 'low') each mapping to a dict of nodes and their corresponding extreme input indices.
            """

            # Calculate mean and standard deviation for each node
            node_stats = {}  # Dictionary to store mean and std for each node
            for _, layers_activations in self.activations_per_input.items():
                for layer_name, activations in layers_activations.items():
                    if layer_name not in node_stats:
                        node_stats[layer_name] = []
                    node_stats[layer_name].append(activations)

            for layer_name, activations_list in node_stats.items():
                node_stats[layer_name] = {
                    'mean': np.mean(activations_list, axis=0),
                    'std': np.std(activations_list, axis=0)
                }

            activations_structure = {}

            # Assuming 'model' is your neural network model
            for name, module in self.model.named_modules():
                # Skip the model itself as it's also returned by named_modules
                if name != "" and name in self.layer_names:
                    activations_structure[name] = {'high': {}, 'low': {}}

            # Identify inputs with extreme activations
            for input_index, layers_activations in self.activations_per_input.items():
                for layer_name, activations in layers_activations.items():
                    if self.expected_outputs[input_index] == label_digit and self.predicted_outputs[input_index] == pred_digit:
                        mean = node_stats[layer_name]['mean']
                        std = node_stats[layer_name]['std']
                        high_threshold = mean + std_dev_threshold * std
                        low_threshold = mean - std_dev_threshold * std

                        high_indices = np.where(activations > high_threshold)[0]
                        low_indices = np.where(activations < low_threshold)[0]

                        for idx in high_indices:
                            activations_structure[layer_name]['high'][idx] = activations_structure[layer_name]['high'].get(idx, 0) + 1
                        for idx in low_indices:
                            activations_structure[layer_name]['low'][idx] = activations_structure[layer_name]['low'].get(idx, 0) + 1
            return activations_structure
            
    def get_previous_layer_name(self, current_layer_name):
        try:
            index = self.layer_names.index(current_layer_name)
        except ValueError:
            print("Layer name not found in the model.")
            return None
        
        if index == 0:
            return None
        else:
            return self.layer_names[index - 1]





In [11]:
explainable_net = ExplainableNetwork(net)

In [12]:
net.train()  # Set the network to training mode

for images, labels in trainloader:
    images, labels = images.to(device), labels.to(device)

    # Zero the parameter gradients
    net.zero_grad()

    # Forward pass
    outputs = net(images)
    
    # Calculate loss
    loss = criterion(outputs, labels)
    
    # Backward pass to compute gradients
    loss.backward()

In [13]:
explainable_net.print_structure()

Explainable Network Structure and Data Summary:

Activations:
Layer conv1: 32 nodes, 60000 data points each
Layer pool: 64 nodes, 60000 data points each
Layer conv2: 64 nodes, 60000 data points each
Layer fc1: 128 nodes, 60000 data points each
Layer fc2: 64 nodes, 60000 data points each
Layer fc3: 10 nodes, 60000 data points each

Gradients:
Layer fc3: 10 nodes, 60000 data points each
Layer fc2: 64 nodes, 60000 data points each
Layer fc1: 128 nodes, 60000 data points each
Layer pool: 32 nodes, 60000 data points each
Layer conv2: 64 nodes, 60000 data points each
Layer conv1: 32 nodes, 60000 data points each


In [14]:
inputs = all_images[12].unsqueeze(0).to(device)  # Add a batch dimension

net.eval()

# Forward pass to get both outputs
with torch.no_grad():
    regular_output, explainable_output = net(inputs, return_explainable=True)

print("Regular Output:", regular_output)
print("Explainable Output:", explainable_output)

Regular Output: tensor([[-16.6261,  -7.4099,  -7.6156,  22.6847, -11.1909,   2.0315, -12.6433,
          -1.7090,   1.4503,  -6.2504]], device='cuda:0')
Explainable Output: tensor(1.2106, device='cuda:0')
